# Generate Training Data

We wish to speed up the process of having a human being name entities that they recognize in a paragraph, and having their positions in the paragraph identified and placed in a syntax usable by spaCy's NER training routine.

## Open PDF file, extract page two and display as sentences

In [1]:
import spacy
import PyPDF2
nlp = spacy.load('en_core_web_sm')

#Open PDF file for reading
pdfFile = open("BarCvDescLJ11.pdf", mode="rb")
pdfReader = PyPDF2.PdfFileReader(pdfFile)

# Select a page to work on
pageNumber = 11

# Get text
OnePage = pdfReader.getPage(pageNumber-1) #0-based count
OnePageText = OnePage.extractText()

# Close PDF file
pdfFile.close()

# Remove newlinesxmx82k-&a. It appears multiple newlines together makes
# Spacy think that is the end of a sentence. The PDF reader reads the text in
# an odd fashion
OnePageText = OnePageText.replace('\n','')

# create a spaCy doc object from the page and break it into sentences
doc = nlp(OnePageText)
l=0
for sent in doc.sents:
     print(l, ": ", sent)
     l = l+1


0 :  11  
1 :  there are few to no barbs on lateral veins.
2 :  The crease is V- shaped.
3 :  Kernels are broad at the center; lateral kernels are moderately twisted.
4 :  At the time of release it was resistant to stem rust, moderately resistant to spot blotch and net blotch, and susceptible to loose smut, false loose smut, covered smut, powdery mildew, and barley stripe mosaic.
5 :  It was evaluated as Entry 690 in the UC Regional Cereal Testing program from 1984-1985 for spring planting in the intermountain region of northern California.
6 :  Crop Science 22:1083 (1982)    
7 :  B1201  B1201 is a two-rowed spring malting barley.
8 :  It was released by Busch Agricultural Resources.
9 :  It is late maturing and mid-tall (similar in plant height to Klages).
10 :  It was evaluated as Entry 770 in the UC Regional Cereal Testing program from 1987-89 for spring planting in the intermountain region of northern California.    
11 :  B1202  
12 :  B1202 is a two-rowed spring malting barley.


## Read in Per-line named entity file and match entities to sentence positions.

In [3]:
import re
import csv
import pandas as pd
fname = "Data/barley_p11_ner.txt"

# Covert the nlp senetence generator into a list of sentences
sentences = list(doc.sents)

# Open the file of manually matched pairs (sentence # <tab> word phrase <tab> named entity)
# e.g.:
#  0      AC Metcalfe     CVAR
#  0      two-rowed       TRAT
#  0      barley          CROP
#  1      Agri-Food Candada   ORG
#  1      1997    DATE
file = open(fname)
reader = csv.reader(file, delimiter='\t', quoting=csv.QUOTE_NONE)
data = list()

for row in reader:
    try:
        (sentIndex, phrase, label) = row
        sent = sentences[int(sentIndex)].string.rstrip()
        
        # find all instances of the 'phrase' in the 'sent'.
        iter = re.finditer(r"\b"+phrase+r"\b", sent)
        indices = [m.start(0) for m in iter]
        
        # check to make sure the phrase the user said was there was indeed found
        if len(indices) == 0:
            raise ValueError
                
        # print out all instances
        for i in indices:
#            print(sentIndex, sent, phrase, "("+str(i), i+len(phrase), "'"+label+"')")
            data.append([sentIndex, sent, phrase, "("+str(i)+", "+str(i+len(phrase))+", '"+label+"')"])
            
    except:
        print("Handle manually: ", row)
        
df = pd.DataFrame(data, columns = ["Index", "Sentence", "Phrase", "MatchInfo"])
print(df)


Handle manually:  ['6', 'Crop Science 22:1083 (1982)', 'JRNL']
    Index                                           Sentence  \
0       1        there are few to no barbs on lateral veins.   
1       2                           The crease is V- shaped.   
2       3  Kernels are broad at the center; lateral kerne...   
3       3  Kernels are broad at the center; lateral kerne...   
4       4  At the time of release it was resistant to ste...   
..    ...                                                ...   
133    64  It is medium maturing and medium height with g...   
134    64  It is medium maturing and medium height with g...   
135    64  It is medium maturing and medium height with g...   
136    65  At the time of evaluation it was moderately su...   
137    65  At the time of evaluation it was moderately su...   

                     Phrase         MatchInfo  
0             lateral veins  (29, 42, 'PLAN')  
1                    crease   (4, 10, 'PLAN')  
2                   Kern

## Create a function to clean up overlapping intervals

In [4]:
import re
coordRegex = re.compile(r'(\d+), (\d+)')

def sortByStart(coords):
    """For use in sort routines, return object with lowest (X,Y) values"""
    # split out coordinates that come in as (5, 7, 'CVAR')
    mo = coordRegex.search(coords)
    return(int(mo.group(1)))

def overlaps(coord1, coord2):
    """Check if coordinates of the form 5, 7, 'CVAR' and 32, 46, 'TRAT' overlap"""
    mo1 = coordRegex.search(coord1)
    mo2 = coordRegex.search(coord2)
    coord1Low = int(mo1.group(1))
    coord1High = int(mo1.group(2))
    coord2Low = int(mo2.group(1))
    coord2High = int(mo2.group(2))
    
    if ((coord1High >= coord2Low) and (coord1Low <= coord2Low) or
        (coord2High >= coord1Low) and (coord2Low <= coord1Low)):
        return True
    else:
        return False

def keepFirst(coord1, coord2):
    """Given overlapping coordinates, return the wider encompassing one."""
    mo1 = coordRegex.search(coord1)
    mo2 = coordRegex.search(coord2)
    coord1Low = int(mo1.group(1))
    coord1High = int(mo1.group(2))
    coord2Low = int(mo2.group(1))
    coord2High = int(mo2.group(2))
 
    if (int(coord1High) - int(coord1Low)) >= (int(coord2High) - int(coord2Low)):
        return True
    else:
        return False

# print("Should be false:", overlaps("(5, 7, 'CVAR')", "(32, 46, 'TRAT')"))
# print("Should be true:", overlaps("(26, 46, 'TRAT')", "(32, 46, 'TRAT')"))
# print("Should be true:", overlaps("(26, 46, 'TRAT')", "(26, 46, 'TRAT')"))
# print("Keeper:", keepFirst("34, 46, 'TRAT'", "34, 46, 'TRAT'"))

def cleanIntervals(inputString=""):
    """order intervals like (5, 7, 'CVAR'), (32, 46, 'TRAT'), (26, 46, 'TRAT') and remove overlapping ones."""
    inputString = inputString.lstrip("(").rstrip(")")
    intervalList = inputString.split("), (")
    intervalList.sort(key = sortByStart)
#    print("Sorted Interval List:", intervalList)

    # Pairwise compare every interval in the list to every other interval to check overlap
    keeperList = [True]*len(intervalList) # Logic array to determine if each interval should be kept
    i=0
    for interval1 in intervalList:
        for interval2 in intervalList:
            if interval1 == interval2:
                if intervalList.index(interval1) != i: # when both are the same we reject the higher one
                    keeperList[i] = False
            else:
                if overlaps(interval1, interval2) and keepFirst(interval1, interval2) == False:
                    keeperList[i] = False
        i = i+1
        
#    print("keeperList:", keeperList)
   
    # Build up the return interval list
    returnStr = "("
    for interval, isKeeper in zip(intervalList, keeperList):
        if isKeeper:
            returnStr = returnStr + interval + "), ("
    return (returnStr.rstrip("), (") + ")")
        
# cleanIntervals("(5, 7, 'CVAR'), (32, 46, 'TRAT'), (5, 9, 'CVAR'), (48, 55, 'ORG'), (26, 46, 'TRAT')")
# cleanIntervals("(0, 8, 'CVAR'), (0, 5, 'CVAR'), (21, 26, 'PLAN'), (32, 37, 'CVAR')")
cleanIntervals("(0, 12, 'CVAR'), (39, 49, 'CVAR'), (39, 49, 'CVAR'), (71, 77, 'CVAR'), (71, 77, 'CVAR'), (92, 113, 'TRAT'), (140, 150, 'CVAR'), (140, 150, 'CVAR'), (181, 187, 'CVAR'), (181, 187, 'CVAR')")


"(0, 12, 'CVAR'), (39, 49, 'CVAR'), (71, 77, 'CVAR'), (92, 113, 'TRAT'), (140, 150, 'CVAR'), (181, 187, 'CVAR')"

## Aggregate all matches for each sentence on a single line and output in spaCy training format

In [5]:
# use Pandas dataframes to aggregate all entity matches together for a single sentence
agg_rules = {'Sentence': 'first', 'Phrase': 'first', 'MatchInfo': lambda x: ', '.join(x)}
res = df.groupby('Index').agg(agg_rules)
#print(res)

# Now format it just like what is needed for the spaCy training module: 
# E.g.:
# ('Eight-Twelve is a six-rowed winter feed barley', {'entities': [(0, 12, 'CVAR'), (18, 27, 'TRAT'), (28, 39, 'TRAT'),(40, 46, 'CROP')]}),
records = res.to_dict('records')
for i in range(0,len(records)):
    print("    ('"+records[i]['Sentence']+"', {'entities': ["+cleanIntervals(records[i]['MatchInfo'])+"]}),")
    #print("    ('"+records[i]['Sentence']+"', {'entities': ["+records[i]['MatchInfo']+"]}),")


    ('there are few to no barbs on lateral veins.', {'entities': [(29, 42, 'PLAN')]}),
    ('B1202 is a two-rowed spring malting barley.', {'entities': [(0, 5, 'CVAR'), (11, 20, 'TRAT'), (21, 27, 'TRAT'), (28, 35, 'TRAT'), (36, 42, 'CROP')]}),
    ('It was released by Busch Agricultural Resources in 1989.', {'entities': [(19, 47, 'ORG'), (51, 55, 'DATE')]}),
    ('It was selected from the cross RDB70-268/2B75-1223//Klages.', {'entities': [(31, 58, 'PED')]}),
    ('Its experimental designation was 2B81-4038.', {'entities': [(33, 42, 'ALAS')]}),
    ('It is medium late maturing and mid-tall (similar in maturity and plant height to Klages).', {'entities': [(18, 26, 'TRAT'), (31, 39, 'TRAT'), (52, 60, 'TRAT'), (65, 77, 'TRAT'), (81, 87, 'CVAR')]}),
    ('It is susceptible to lodging but has higher yield potential than Klages.', {'entities': [(21, 28, 'TRAT'), (44, 59, 'TRAT'), (65, 71, 'CVAR')]}),
    ('Spikes are lax.', {'entities': [(0, 6, 'PLAN')]}),
    ('Awns are rough.', {'entities':